# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
# подключаем библиотеку pandas для работы с таблицами
import pandas as pd
from IPython.display import display
# читаем данные из файла .csv с помощью метода read_csv() 
df = pd.read_csv('/datasets/data.csv')
# выводим первые 5 строк таблицы с помощью метода head() 
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# с помощью метода info() изучим структуру таблицы: Типы данных, колличество строк, столбцов, пропущенных данных.
 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

Таблица данных состоит из 21525 строк и 12 столбцов. Данные пропущены в столбцах days_employed(общий трудовой стаж в днях) и total_income(ежемесячный доход). Причём колличество пропущенных данных одинаково

In [3]:
# сохраним в переменной missed_days_employed все строки с пропущенным значением 'days_employed'
missed_days_employed = df.loc[df['days_employed'].isnull()]
print('Колличество строк с пропущенным трудовым стажем:', len(missed_days_employed))
# проверим, что для всех этих строк значения в 'total_income' также пропущены
display(missed_days_employed.loc[missed_days_employed['total_income'].isnull() == False])

Колличество строк с пропущенным трудовым стажем: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Около 10% - такую долю составляют пропущенные значения в каждом из столбцов с пропусками.
Причиной возникновения пропусков неизвестны, но возможно это человеческий фактор при выгрузке или записи данных.

In [4]:
#найдем  медиану дохода и стажа соотв.
total_income_median = df['total_income'].median()
days_employed_median = df['days_employed'].median()

#заменим отрицательные числа на положительные в стаже
def test(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(test)
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
# с помощью метода loc[] и логической операции отберем записи с 'days_employed' > 30000 и выведем колличество записей с помощью функции len()
wrong_days_employed = len(df.loc[df['days_employed'] > 30000])
print('Колличество строк с неверным трудовым стажем:', wrong_days_employed)
# аналогично найдем колличество строк с отрицательным тррудовым стажем.
neg_days_employed = len(df.loc[df['days_employed'] < 0])
print('Колличество строк с отрицательным трудовым стажем:', wrong_days_employed)

Колличество строк с неверным трудовым стажем: 3445
Колличество строк с отрицательным трудовым стажем: 3445


Колличество трудовых дней может быть только целым числом.
Преобразуем типы данных в столбцt days_employed c float64 на целочисленный int64.
Преобразование проведем методом astype(). Он произведет изменение типа для вещественных чисел и выдаст ошибку, если данные изначально были введены некорректно (например, если встретятся данные в виде строк вместо вещественных чисел)

In [6]:
# типы данных преобразуем методом .astype(), с помощью try-except отслеживаем некорректные типы данных
try:
    df['days_employed'] = df['days_employed'].astype('int64')
except:
    print('Некорректный тип данных. Пожалуйста, убедитесь, что данные хранятся в виде вещественных чисел')

### Шаг 2.3. Изменение типов данных.

In [7]:
#Замените вещественный тип данных в столбце total_income на целочисленный
df['total_income'] = df['total_income'].astype('int64')

### Шаг 2.4. Удаление дубликатов.

Для поиска дубликатов сперва преобразуем все строковые типы данных к нижнему регистру.
Затем посчитаем колличество дублированных строк и выведем их для анализа.

In [8]:
# для преобразования к нижнему регистру используем метод str.lower() затем запишем преобразованные данные в таблицу
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
print('Колличество дублированных данных: ', df.duplicated().sum())

Колличество дублированных данных:  71


In [9]:
display(df.loc[df.duplicated()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,-1203,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
3290,0,-1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
4182,1,-1203,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,-1203,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,-1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
6312,0,-1203,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017,строительство жилой недвижимости
7808,0,-1203,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7921,0,-1203,64,высшее,0,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
7938,0,-1203,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,-1203,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование


Все дублированные данные являются строками, в которых ранее были обработаны пропущенные значения. Поскольку в таблице нет уникальных идентификаторов клиентов, невозможно точно отверждать: данные дублированы или информация по разным клиентам совпала случайным образом.
Возможные причины появления дубликатов: повторное заполнение или сохранение данных о клиенте в базе данных, сбой в системе при копировании/выгрузке базы данных.

In [10]:
#Удалим дублированные строки в учебных целях. Для этого используем метод drop_duplicates() и восстановим индексы после удаления строк методом reset_index()
df = df.drop_duplicates().reset_index(drop = True)
print('Колличество строк после удаления дубликатов:', len(df))

Колличество строк после удаления дубликатов: 21454


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [11]:
#создаём новые датафреймы
family_status_dict = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_status_dict)
education_id_df = df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
display(education_id_df)




,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [12]:
# из основного датафрейма удаляем поля family_status и education
df = df.drop(['family_status', 'education'], axis = 1)
display(df.head(10))



,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

In [13]:
#создаём столбец total_income_category
def income_level_func(row):
    
    inc = row['total_income']
    
    if inc <= 30000:
        return 'E'
    elif inc <= 50000:
        return 'D'
    elif inc <= 200000:
        return 'C'
    elif inc <= 1000000:
        return 'B'
    else:
        return 'A'

df['total_income_category'] = df.apply(income_level_func, axis=1)
display(df.head(10))


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

In [14]:
#создаём функцию, которая на основании данных из столбца pupose сормирует новый столбец purpose_category
def set_purpose_cat(purpose_category):
    if 'свад' in purpose_category:
        return 'свадьба'
    elif 'ремонт' in purpose_category:
        return 'ремонт'
    elif 'авто' in purpose_category:
        return 'автомобиль'
    elif 'образ' in purpose_category:
        return 'образование'
    elif 'жиль' in purpose_category or 'недвиж' in purpose_category:
        return 'недвижимость'
    else:
        return 'другое'
    
#Применим функцию:

df['purpose_catagory'] = df['purpose'].apply(set_purpose_cat)


### Ответы на вопросы.

#Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок.

In [15]:
# Категоризируем колличество детей в семье.
# создадим словарь для категорий колличества детей в семье children_dict
children_data = {'children_cat_id': [0, 1, 2, 3, 4],
                 'children_cat_name': ['Нет детей', 'Один ребенок', 'Двое детей', 
                                       'Многодетная семья', 'Статус неизвестен']}
children_columns = ['children_cat_id', 'children_cat_name']
children_dict = pd.DataFrame(data = children_data, columns = children_columns)
display(children_dict)

,children_cat_id,children_cat_name
0,0,Нет детей
1,1,Один ребенок
2,2,Двое детей
3,3,Многодетная семья
4,4,Статус неизвестен


In [16]:
def set_children_cat_id(children):
    """
    input: children (int64) - целое число, описывает колличество детей в семье
    output: (int64) - целое число, идентификатор категории колличества детей в семье
    Функция сопоставляет колличеству детей в семье одну из категорий:
    
    Children:  ID:   Категория:
    0          0     Нет детей
    1          1     Один ребенок
    2          2     Двое детей
    >2         3     Многодетная семья
    <0         4     Статус неизвестен (значение -1 в столбце 'children')
    """
    
    if children == 0:
        return 0
    elif children == 1:
        return 1
    elif children == 2:
        return 2
    elif children > 2:
        return 3
    else:
        return 4

# с помощью метода apply применяем функцию set_children_cat_id ко всем 'children' 
# в таблице 'df'
df['children_cat_id'] = df['children'].apply(set_children_cat_id)
display(df.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_catagory,children_cat_id
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,недвижимость,1
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,автомобиль,1
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,недвижимость,0
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,образование,3
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,свадьба,0
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,недвижимость,0
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,недвижимость,0
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,образование,0
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,свадьба,2
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,недвижимость,0


Для ответа на вопрос составим новую таблицу children_debt с данными о задолженностях по кредиту debt из таблицы df и информацией о колличестве детей children_cat_id, children_cat_name из словаря children_dict. Далее для наглядности результаты выведем в сводную таблицу.
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.

In [17]:
children_debt = df.merge(children_dict, on='children_cat_id', how='left')[['children_cat_id', 
                                                                             'children_cat_name', 
                                                                             'debt']]
# сводную таблицу получим методом pivot_table(). 
children_debt_pivot = children_debt.pivot_table(index = 'children_cat_name', columns = 'debt', 
                                                values = 'children_cat_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
children_debt_pivot['rate'] = children_debt_pivot[1] / (children_debt_pivot[0] + children_debt_pivot[1])
display(children_debt_pivot)

debt,0,1,rate
children_cat_name,,,
Двое детей,1858,194,0.094542
Многодетная семья,417,39,0.085526
Нет детей,13028,1063,0.075438
Один ребенок,4364,444,0.092346
Статус неизвестен,46,1,0.021277


Вывод:

Минимальная доля клиентов с задолженностями среди группы без детей (7.5%).
Среди клиентов с одним и двумя детьми данные мало отличаются, примерно 9.5% клиентов имеют задолженности.
Среди многодетных семей доля клиентов с задолженностями составляет около 8.5%. Но в данной группе сравнительно небольшая выборка и необходимо дальнейшее иследование относительно достоверности результатов.
В целом доля задолженностей по кредитам среди клиентов не сильно меняется от колличества детей в семье (в пределах 1-2%).

In [18]:
#Вопрос2: Есть ли зависимость межлу семейным положением и возвратом кредита.

Для ответа на вопрос составим новую таблицу family_status_debt с данными о задолженностях по кредиту debt из таблицы df и информацией о семейном статусе family_status_id, family_status из словаря family_status_dict.
Далее для наглядности результаты выведем в сводную таблицу.
Также рассчитаем отношение колличества клиентов с задолженностями к общему колличеству клиентов по каждой группе.

In [19]:
family_status_debt = df.merge(family_status_dict, 
                                on='family_status_id', how='left')[['family_status_id',
                                                                   'family_status',
                                                                   'debt']]
# сводную таблицу получим методом pivot_table(). 
family_status_pivot = family_status_debt.pivot_table(index = 'family_status', columns = 'debt',
                                                     values = 'family_status_id', aggfunc = 'count')
# найдем в каждой группе долю клиентов с задоленностями относительно 
# общего числа клиентов в данной группе
family_status_pivot['rate'] = family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1])
display(family_status_pivot)

debt,0,1,rate
family_status,,,
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3763,388,0.093471
женат / замужем,11408,931,0.075452
не женат / не замужем,2536,274,0.097509


Вывод:

Минимальная доля клиентов с задолженностями среди группы вдовец / вдова (6.5%).
Среди клиентов групп не женат / не замужем и гражданский брак данные мало отличаются, примерно 9.5% клиентов имеют задолженности. Это максимальные показатели среди всех групп
Среди клиентов группы вдовец / вдова доля задолженностей минимальна и составляет около 6.5%. Но в данной группе сравнительно небольшая выборка и необходимо дальнейшее иследование относительно достоверности результатов.
Среди клиентов в категориях женат / замужем и в разводе доля задолженностей примерно одинакова и составляет 7-7.5%. Но выборка клиентов в разводе сравнительно неболшая, и данные могут быть недостоверны.
Судя по полученным данным, семейные клиенты имеют меньше задолженностей по кредитам, чем неженатые/незамужние клиенты или в гражданском браке. Возможно, семейные пары более тщательно планируют свой бюджет и серьезнее относятся к своим обязательствам.
В целом доля задолженностей по кредитам среди клиентов не сильно меняется от семейного положения (в пределах 3%).

In [20]:
#Вопрос3: Если зависимость между уровнем дохода и возвратом кредита в срок.

In [21]:
total_pivot = df.pivot_table(index='total_income_category', values='debt', aggfunc=['mean'])
display(total_pivot)

,mean
,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084915
D,0.060000
E,0.090909


Вывод:

Клиенты с высоким уровнем доходов (> 230к) в целом отдают кредиты в срок чаще (7% клиентов с задолженностями).
Клиенты с более низкими доходами (до 90к), а также с доходами в 175к-230к в среднем имеют задолженности в 8% случаев. Это может быть связано с нехваткой финансов или проблемами с планированием бюджета.
Клиенты со средними доходами (90к - 175к) имеют задолженности в 8-9% случаев. Это может быть связано с проблемами в планировании бюджета (большие сумммы кредитов и маленькие сроки могут приводить к высоким ежемесячным выплатам, с которыми клиенты не всегда справляются).
В целом данные не сильно отличаются с изменением уровня доходов (в пределах 2%).

In [22]:
#Вопрос4: Как разные цели кредита влияют на его возврат в срок.

In [23]:
df_pivot = df.pivot_table(index='purpose_catagory', values='debt', aggfunc=['mean'])
display(df_pivot)


,mean
,debt
purpose_catagory,
автомобиль,0.093590
недвижимость,0.073207
образование,0.092200
ремонт,0.057661
свадьба,0.080034


Вывод:

Кредиты на ремонт квартиры в среднем задерживают в 5.7% случаев. Но выборка в этой категории относительно мала, сложно судить о достоверности результатов.
Больше всего кредитов берут на покупку и другие операции с недвижимостью. Задолженности возникают у 7.3% клиентов.
Кредиты на образование и покупку автомобиля задерживают в 9.2-9.3% случаев (на 2% больше чем кредиты, связанные с недвижимостью). На данном этапе сложно определить причины такой разницы.
В целом данные по задолженностям не сильно меняются от целей, на котоые взяты кредиты (в пределах 3 - 3.5%).

## Общий вывод:

В Работе были проанализированы данные о статистике платежеспособности клиентов.  Среди целей на получение кредита было выделено 5 основных групп: свадьба, ремонт, автомобиль, образование, недвижимость.Были проанализированы задолженности клиентов по кредитам в зависимости от количества детей в семье, семейного положения, уровня месячного дохода и цели кредита.
Данные выведены в сводные таблицы. Дополнительно рассчитаны доли клиентов с задолженностями для каждой категории.
В некоторых категориях выборки оказались сравнительно малы, что не позволяет делать однозначные выводы о результатах исследований. Возможные пути решения проблемы - наращивать базу данных.
По всем категориям доли клиентов с задолженностями не превышала 10%, а различия значений между отдельными группами разделения не превышали 3.5%.